In [1]:
import numpy as np
import pandas as pd
import random
from docplex.mp.model import Model
from tabulate import tabulate

# parameters
I, J, L, K = 3, 10, 10, 3  # I is unused; J = med shops, L = food shops, K = truck types
Q = [13.5, 15, 18]  # truck capacities by type
e = [1339, 1330, 1300]  # per-unit delivery cost to medical shops for each truck type
f = [1628, 1480, 1563]  # per-unit delivery cost to food shops for each truck type
THC = [2000, 2500, 2800]  # fixed truck hiring cost per truck type
DHC = 10000  # fixed cost to hire a driver
T_max = 8  # max time a truck can operate in a trip
service_time = 0.25  # fixed time spent at each shop (in hours)
travel_time_per_unit = 0.1  # additional time added per unit of demand

# taguchi L16 design of experiments for tuning ga parameters
# each tuple = (population size, crossover prob, mutation prob, iterations)
l16 = [
 (120,0.4,0.04,50),(120,0.5,0.06,100),(120,0.6,0.08,150),(120,0.3,0.10,200),
 (60,0.4,0.06,150),(60,0.5,0.04,200),(60,0.6,0.10,50),(60,0.3,0.08,100),
 (80,0.4,0.08,200),(80,0.5,0.10,150),(80,0.6,0.04,100),(80,0.3,0.06,50),
 (100,0.4,0.10,100),(100,0.5,0.08,50),(100,0.6,0.06,200),(100,0.3,0.04,150)
]

# generate random demand between 5 and 14 units for each medical and food shop
demand_med = np.random.randint(5, 15, size=J)
demand_food = np.random.randint(5, 15, size=L)

# ------------------ cplex milp optimization ------------------
def solve_cplex(d_med, d_food):
    mdl = Model(name="HTFO_CPLEX")  # create a cplex model

    # xjk[j][k][t] = 1 if medical shop j is served by truck type k in trip t
    xjk = [[[mdl.binary_var(name=f"xjk_{j}_{k}_{t}") for t in range(10)] for k in range(K)] for j in range(J)]
    # xlk[l][k][t] = 1 if food shop l is served by truck type k in trip t
    xlk = [[[mdl.binary_var(name=f"xlk_{l}_{k}_{t}") for t in range(10)] for k in range(K)] for l in range(L)]
    # ykt[k][t] = 1 if truck type k is active in trip t (i.e., driver hired)
    ykt = [[mdl.binary_var(name=f"ykt_{k}_{t}") for t in range(10)] for k in range(K)]

    # each medical shop must be assigned to exactly one truck-trip combo
    for j in range(J):
        mdl.add_constraint(mdl.sum(xjk[j][k][t] for k in range(K) for t in range(10)) == 1)
    # each food shop must be assigned to exactly one truck-trip combo
    for l in range(L):
        mdl.add_constraint(mdl.sum(xlk[l][k][t] for k in range(K) for t in range(10)) == 1)

    total_cost = 0  # initialize total cost expression

    # loop over truck types and trips to accumulate time, load, and cost
    for k in range(K):
        for t in range(10):
            time_expr = 0  # total time used by this truck in this trip
            load_expr = 0  # total load carried in this trip

            for j in range(J):
                u = d_med[j]
                time_expr += (service_time + travel_time_per_unit * u) * xjk[j][k][t]
                load_expr += u * xjk[j][k][t]
                total_cost += e[k] * u * xjk[j][k][t]  # cost for delivering to medical shops
                mdl.add_constraint(ykt[k][t] >= xjk[j][k][t])  # activate truck if used

            for l in range(L):
                u = d_food[l]
                time_expr += (service_time + travel_time_per_unit * u) * xlk[l][k][t]
                load_expr += u * xlk[l][k][t]
                total_cost += f[k] * u * xlk[l][k][t]  # cost for delivering to food shops
                mdl.add_constraint(ykt[k][t] >= xlk[l][k][t])  # activate truck if used

            mdl.add_constraint(time_expr <= T_max)  # time limit for each truck-trip
            mdl.add_constraint(load_expr <= Q[k])   # capacity constraint for each truck

            total_cost += THC[k] * ykt[k][t]  # fixed truck hire cost

    # count total drivers used (one per truck-trip where ykt is 1)
    driver_count = mdl.sum(ykt[k][t] for k in range(K) for t in range(10))
    mdl.minimize(total_cost + DHC * driver_count)  # total cost includes driver cost

    sol = mdl.solve()
    return mdl.objective_value if sol else None  # return optimal cost or None if infeasible

# ------------------ genetic algorithm cost function ------------------
def ga_cost(assign, d_med, d_food):
    total = 0
    time_viol_penalty = 1e6  # penalty for exceeding time
    load_viol_penalty = 1e6  # penalty for exceeding load
    time_per_driver = [0 for _ in range(K * 10)]
    load_per_driver = [0 for _ in range(K * 10)]
    assign_count = [0] * (K * 10)

    for j, (k, t) in enumerate(assign[:J]):
        idx = k * 10 + t
        total += e[k] * d_med[j]
        time_per_driver[idx] += service_time + travel_time_per_unit * d_med[j]
        load_per_driver[idx] += d_med[j]
        assign_count[idx] += 1

    for l, (k, t) in enumerate(assign[J:]):
        idx = k * 10 + t
        total += f[k] * d_food[l]
        time_per_driver[idx] += service_time + travel_time_per_unit * d_food[l]
        load_per_driver[idx] += d_food[l]
        assign_count[idx] += 1

    for i in range(K * 10):
        if assign_count[i] > 0:
            total += DHC + THC[i // 10]  # add driver and truck cost
        if time_per_driver[i] > T_max:
            total += time_viol_penalty
        if load_per_driver[i] > Q[i // 10]:
            total += load_viol_penalty

    return total

# ------------------ run the genetic algorithm ------------------
def run_ga(d_med, d_food, pop, cxpb, mutpb, gen_max):
    n = J + L  # total number of shops (10 medical + 10 food = 20)

    # initialize the population with random individuals
    # each individual is a list of n tuples (k, t) representing truck type and trip number
    popu = [[(random.randint(0, K-1), random.randint(0, 9)) for _ in range(n)] for _ in range(pop)]

    # loop for the specified number of generations
    for _ in range(gen_max):
        # calculate fitness (total cost + penalties) for each individual in the population
        fitness = [ga_cost(ind, d_med, d_food) for ind in popu]

        new_pop = []  # new generation

        # generate new population of size pop using selection, crossover and mutation
        while len(new_pop) < pop:
            # tournament selection to pick parent 1
            a, b, c = random.sample(range(pop), 3)
            # pick the best of the three based on lowest fitness
            p1 = popu[min([(fitness[a], a), (fitness[b], b), (fitness[c], c)])[1]]

            # tournament selection to pick parent 2
            a, b, c = random.sample(range(pop), 3)
            p2 = popu[min([(fitness[a], a), (fitness[b], b), (fitness[c], c)])[1]]

            # perform crossover with probability cxpb
            if random.random() < cxpb:
                pt = random.randint(1, n - 2)  # choose a random crossover point
                # children are formed by swapping segments after the crossover point
                c1, c2 = p1[:pt] + p2[pt:], p2[:pt] + p1[pt:]
            else:
                # if no crossover, children are just clones of parents
                c1, c2 = p1[:], p2[:]

            # perform mutation on each child
            for c in (c1, c2):
                for i in range(n):
                    # with probability mutpb, mutate the gene by assigning a new truck-trip combo
                    if random.random() < mutpb:
                        c[i] = (random.randint(0, K-1), random.randint(0, 9))
                # add the mutated child to the new population
                new_pop.append(c)

        # update population with the newly generated population (same size)
        popu = new_pop[:pop]

    # after all generations, calculate final fitness
    final_fitness = [ga_cost(ind, d_med, d_food) for ind in popu]
    idx = np.argmin(final_fitness)  # find the index of the best solution
    return final_fitness[idx], popu[idx]  # return the best cost and the best assignment


# ------------------ run all optimization and comparisons ------------------
cplex_cost = solve_cplex(demand_med, demand_food)
print(f"\n[CPLEX] Optimal cost: {cplex_cost:.2f}" if cplex_cost else "\n[CPLEX] No feasible solution")

# evaluate each ga configuration from taguchi L16
l16_results = []
for pop, cx, mut, it in l16:
    costs = []
    for seed in range(3):
        np.random.seed(42 + seed)  # reproducibility
        d_med = np.random.randint(5, 15, size=J)
        d_food = np.random.randint(5, 15, size=L)
        cost, _ = run_ga(d_med, d_food, pop, cx, mut, it)
        costs.append(cost)
    avg_cost = np.mean(costs)
    l16_results.append((pop, cx, mut, it, avg_cost))

# show results in table form
l16_df = pd.DataFrame(l16_results, columns=['population', 'crossover', 'mutation', 'iterations', 'avg_cost'])
print("\n[L16 TABLE] GA average cost per configuration:")
print(tabulate(l16_df, headers='keys', tablefmt='github'))

# select the best configuration from L16
best_params = l16_df.sort_values('avg_cost').iloc[0]
print(f"\n[SELECTED GA CONFIG] Best GA config: {best_params.to_dict()}")

# re-run ga using best parameters on actual demand set
final_cost, final_assign = run_ga(demand_med, demand_food,
                                  int(best_params['population']),
                                  best_params['crossover'],
                                  best_params['mutation'],
                                  int(best_params['iterations']))

# final comparison against cplex
if cplex_cost:
    print(f"\n[FINAL GA] Cost with best L16 config: {final_cost:.2f}")
    print(f"[COMPARISON] GA cost = {final_cost:.2f}, CPLEX cost = {cplex_cost:.2f}, Deviation = {(final_cost - cplex_cost)/cplex_cost*100:.2f}%")
else:
    print(f"\n[FINAL GA] Cost with best L16 config: {final_cost:.2f}")
    print("[COMPARISON] CPLEX was infeasible.")




[CPLEX] Optimal cost: 392043.00

[L16 TABLE] GA average cost per configuration:
|    |   population |   crossover |   mutation |   iterations |   avg_cost |
|----|--------------|-------------|------------|--------------|------------|
|  0 |          120 |         0.4 |       0.04 |           50 |     431661 |
|  1 |          120 |         0.5 |       0.06 |          100 |     429704 |
|  2 |          120 |         0.6 |       0.08 |          150 |     444194 |
|  3 |          120 |         0.3 |       0.1  |          200 |     452290 |
|  4 |           60 |         0.4 |       0.06 |          150 |     424654 |
|  5 |           60 |         0.5 |       0.04 |          200 |     741163 |
|  6 |           60 |         0.6 |       0.1  |           50 |     468761 |
|  7 |           60 |         0.3 |       0.08 |          100 |     446734 |
|  8 |           80 |         0.4 |       0.08 |          200 |     442222 |
|  9 |           80 |         0.5 |       0.1  |          150 |     4545